In [1]:
import nest_asyncio
nest_asyncio.apply()
import pydra

In [2]:
import toimportfunctions

In [3]:
from os import getenv
from pathlib import Path

clinica_data_ci_dir = "/Users/matthieu.joulot/Documents/DWI/ci-complete"

pipeline_dir = Path(clinica_data_ci_dir) / "DWIPreprocessingUsingT1"
pipeline_in_dir = pipeline_dir / "in"
pipeline_ref_dir = pipeline_dir / "ref"
pipeline_out_dir = pipeline_dir / "out"
working_dir = "/Users/matthieu.joulot/Desktop/working_directory"

In [4]:
from nipype.interfaces.mrtrix3.preprocess import DWIBiasCorrect

from pydra.mark import annotate, task

In [5]:
from nipype.interfaces.io import BIDSDataGrabber

bids_data_grabber = BIDSDataGrabber(
    outfields=["t1w", "dwi", "dwi_json", "bvec", "bval"],
    output_query={
        "t1w": {
            "datatype": "anat",
            "suffix": "T1w",
            "extension": ".nii.gz",
        },
        "dwi": { "datatype": "dwi", "suffix": "dwi", "extension": ".nii.gz"},
        "dwi_json": { "datatype": "dwi", "suffix": "dwi", "extension": ".json"},
        "bvec": { "datatype": "dwi", "suffix": "dwi", "extension": ".bvec"},
        "bval": { "datatype": "dwi", "suffix": "dwi", "extension": ".bval"},
    }
)

In [6]:
from nipype.interfaces.fsl.preprocess import BET

bet_2 = BET(frac=0.3,robust=True)

In [7]:
@task
@annotate({"return": {"image_id": str, "t1w": str, "dwi": str, "bvec": str, "bval": str, "total_readout_time": str,"phase_encoding_direction": str}})
def init_input_node(t1w, dwi, bvec, bval, dwi_json):
    """Initialize the pipeline."""
    
    # Extract image ID
    image_id = toimportfunctions.get_subject_id(t1w)

    # Check that the number of DWI, bvec & bval are the same
    toimportfunctions.check_dwi_volume(dwi, bvec, bval)

    # Read metadata from DWI JSON file:
    [total_readout_time, phase_encoding_direction] = toimportfunctions.extract_metadata_from_json(
        dwi_json, ["TotalReadoutTime", "PhaseEncodingDirection"]
    )
    phase_encoding_direction = toimportfunctions.bids_dir_to_fsl_dir(phase_encoding_direction)

    # Print begin message
    toimportfunctions.print_begin_image(
        image_id,
        ["TotalReadoutTime", "PhaseEncodingDirection"],
        [str(total_readout_time), phase_encoding_direction],
    )

    return (
        image_id,
        t1w,
        dwi,
        bvec,
        bval,
        total_readout_time,
        phase_encoding_direction,
    )

In [8]:
@task
@annotate({"return": {"out_reference_b0": str, "out_b0_dwi_merge":str, "out_updated_bval":str, "out_updated_bvec":str}})
def prepare_reference_b0_1(in_dwi, in_bval, in_bvec, low_bval=5, working_directory=working_dir):
    """Prepare reference b=0 image.

    This function prepare the data for further corrections. It co-registers the B0 images
    and then average it in order to obtain only one average B0 images.

    Args:
        in_dwi (str): Input DWI file.
        in_bvec (str): Vector file of the diffusion directions of the DWI dataset.
        in_bval (str): B-values file.
        low_bval (optional, int): Set b<=low_bval such that images are considered b0. Defaults to 5.
        working_directory (str): Temporary folder results where the results are stored. Defaults to None.

    Returns:
        out_reference_b0 (str): Average of the B0 images or the only B0 image.
        out_b0_dwi_merge (str): Average of B0 images merged to the DWIs.
        out_updated_bval (str): Updated gradient values table.
        out_updated_bvec (str): Updated gradient vectors table.
    """
    import hashlib
    import os
    import tempfile


    # Count the number of b0s
    nb_b0s = toimportfunctions.count_b0s(in_bval=in_bval, low_bval=low_bval)

    # Split dataset into two datasets: the b0 and the b>low_bval datasets
    [extracted_b0, out_split_dwi, out_split_bval, out_split_bvec] = toimportfunctions.b0_dwi_split(
        in_dwi=in_dwi, in_bval=in_bval, in_bvec=in_bvec, low_bval=low_bval
    )
    
    
    if nb_b0s == 1:
        # The reference b0 is the extracted b0
        # cprint('Only one b0 for %s' % in_dwi)
        out_reference_b0 = extracted_b0
    elif nb_b0s > 1:
        if working_directory is None:
            working_directory = tempfile.mkdtemp()
        tmp_dir = os.path.join(
            working_directory, hashlib.md5(in_dwi.encode()).hexdigest()
        )
        b0_flirt_results = toimportfunctions.build_n_run_flirt(nb_b0s, extracted_b0, tmp_dir)
        print("b0_flirt_results: ", b0_flirt_results)
        out_reference_b0 = toimportfunctions.b0_average(in_file=b0_flirt_results[1].get_output_field('out_file'))
        print("out_reference_b0: ", out_reference_b0)
    else:
        raise ValueError(
            f"The number of b0s should be strictly positive (b-val file: {in_bval})."
        )
    # Merge datasets such that bval(DWI) = (0 b1 ... bn)
    [out_b0_dwi_merge, out_updated_bval, out_updated_bvec] = toimportfunctions.insert_b0_into_dwi(
        in_b0=out_reference_b0,
        in_dwi=out_split_dwi,
        in_bval=out_split_bval,
        in_bvec=out_split_bvec,
    )
    return out_reference_b0, out_b0_dwi_merge, out_updated_bval, out_updated_bvec

In [9]:
@task
@annotate({"return":{"out_acq": str}})
def generate_acq_file(
    in_dwi, fsl_phase_encoding_direction, total_readout_time, image_id=None
):
    """Generate [`image_id`]_acq.txt file for FSL eddy command.

    Args:
        in_dwi (str): DWI file.
        fsl_phase_encoding_direction (str): PhaseEncodingDirection from BIDS specifications in FSL format (i.e. x/y/z instead of i/j/k).
        total_readout_time (str): TotalReadoutTime from BIDS specifications.
        image_id (str, optional): Optional prefix. Defaults to None.

    Returns:
        out_acq: [`image_id`]_acq.txt or acq.txt file.
    """
    import os

    import nibabel as nb
    import numpy as np

    if image_id:
        out_acq = os.path.abspath(f"{image_id}_acq.txt")
    else:
        out_acq = os.path.abspath("acq.txt")
    vols = nb.load(in_dwi).get_fdata().shape[-1]
    arr = np.ones([vols, 4])
    for i in range(vols):
        if fsl_phase_encoding_direction == "y-":
            arr[i, :] = np.array((0, -1, 0, total_readout_time))
        elif fsl_phase_encoding_direction == "y":
            arr[i, :] = np.array((0, 1, 0, total_readout_time))
        elif fsl_phase_encoding_direction == "x":
            arr[i, :] = np.array((1, 0, 0, total_readout_time))
        elif fsl_phase_encoding_direction == "x-":
            arr[i, :] = np.array((-1, 0, 0, total_readout_time))
        elif fsl_phase_encoding_direction == "z":
            arr[i, :] = np.array((0, 1, 0, total_readout_time))
        elif fsl_phase_encoding_direction == "z-":
            arr[i, :] = np.array((0, 0, -1, total_readout_time))
        else:
            raise RuntimeError(
                f"FSL PhaseEncodingDirection (found value: {fsl_phase_encoding_direction}) "
                f"is unknown, it should be a value in (x, y, z, x-, y-, z-)"
            )

    np.savetxt(out_acq, arr, fmt="%d " * 3 + "%f")

    return out_acq

In [10]:
@task
@annotate({"return":{"out_index": str}})
def generate_index_file(in_bval, low_bval=5.0, image_id=None):
    """Generate [`image_id`]_index.txt file for FSL eddy command.

    Args:
        in_bval (str): Bval file.
        low_bval (float): Define the b0 volumes as all volume bval <= low_bval. Default to 5.0.
        image_id (str, optional): Optional prefix. Defaults to None.

    Returns:
        out_index: [`image_id`]_index.txt or index.txt file.
    """
    import os

    import numpy as np

    assert os.path.isfile(in_bval)
    bvals = np.loadtxt(in_bval)
    idx_low_bvals = np.where(bvals <= low_bval)
    b0_index = idx_low_bvals[0].tolist()

    if not b0_index:
        raise ValueError(
            f"Could not find b-value <= {low_bval} in bval file ({in_bval}). Found values: {bvals}"
        )

    if image_id:
        out_index = os.path.abspath(f"{image_id}_index.txt")
    else:
        out_index = os.path.abspath("index.txt")

    vols = len(bvals)
    index_list = []
    for i in range(0, len(b0_index)):
        if i == (len(b0_index) - 1):
            index_list.extend([i + 1] * (vols - b0_index[i]))
        else:
            index_list.extend([i + 1] * (b0_index[i + 1] - b0_index[i]))
    index_array = np.asarray(index_list)
    try:
        len(index_list) == vols
    except ValueError:
        raise ValueError(
            "It seems that you do not define the index file for FSL eddy correctly!"
        )
    np.savetxt(out_index, index_array.T)

    return out_index


In [11]:
from nipype.interfaces.fsl.epi import Eddy

use_cuda=False
initrand=False
eddy = Eddy(repol=True, use_cuda=use_cuda, initrand=initrand)

In [12]:
## Workflow definition
from pydra import Workflow
from pydra.tasks.nipype1.utils import Nipype1Task

workflow = Workflow(
    name = "dwi_preprocessing_using_t1",
    input_spec = ["input_dir"],
    input_dir = str(pipeline_in_dir / "bids"))



#bids_data_grabber
workflow.add(
    Nipype1Task(
        name="bids_data_grabber", 
        interface=bids_data_grabber, 
        base_dir=workflow.lzin.input_dir
    )
)

#init_input_node
workflow.add(
    init_input_node(
        name="init_input_node",
        interface=init_input_node, 
        t1w=workflow.bids_data_grabber.lzout.t1w,
        dwi=workflow.bids_data_grabber.lzout.dwi,
        dwi_json=workflow.bids_data_grabber.lzout.dwi_json,
        bvec=workflow.bids_data_grabber.lzout.bvec,
        bval=workflow.bids_data_grabber.lzout.bval,
    )
)
workflow.init_input_node.split(["t1w", "dwi", "dwi_json", "bvec", "bval"])

workflow.add(
    prepare_reference_b0_1(
        name="prepare_reference_b0_1",
        interface=prepare_reference_b0_1,
        in_dwi=workflow.init_input_node.lzout.dwi,
        in_bval=workflow.init_input_node.lzout.bval,
        in_bvec=workflow.init_input_node.lzout.bvec,
        low_bval=5,
        working_directory=working_dir,
    )
)
workflow.add(
    Nipype1Task(
        name="bet_2",
        interface=bet_2,
        in_file=workflow.prepare_reference_b0_1.lzout.out_reference_b0,
        mask=True,
    )
)

## Eddy pipeline parts: generate_acq, generate_index, eddy
workflow.add(
    generate_acq_file(
        name="generate_acq_file",
        interface=generate_acq_file,
        in_dwi=workflow.prepare_reference_b0_1.lzout.out_b0_dwi_merge, 
        fsl_phase_encoding_direction=workflow.init_input_node.lzout.phase_encoding_direction,
        total_readout_time=workflow.init_input_node.lzout.total_readout_time,
        image_id=None
    )
)
workflow.add(
    generate_index_file(
        name="generate_index_file",
        interface=generate_index_file,
        in_bval=workflow.prepare_reference_b0_1.lzout.out_updated_bval,
        low_bval=5.0,
        image_id=None
    )
)
workflow.add(
    Nipype1Task(
        name="eddy",
        interface=eddy,
        in_file=workflow.prepare_reference_b0_1.lzout.out_b0_dwi_merge,
        in_mask=workflow.bet_2.lzout.mask_file,
        in_bval=workflow.prepare_reference_b0_1.lzout.out_updated_bval,
        in_bvec=workflow.prepare_reference_b0_1.lzout.out_updated_bvec,
        in_acqp=workflow.generate_acq_file.lzout.out_acq,
        in_index=workflow.generate_index_file.lzout.out_index,
    )
)
## eddy pipeline end

##epi pipeline

##epi pipeline end
workflow.set_output([
    #("out_ref_b0_bet", workflow.bet.lzout.mask_file)
    #("out_reference_b0", workflow.prepare_reference_b0_1.lzout.out_reference_b0), 
    #("out_b0_dwi_merge", workflow.prepare_reference_b0_1.lzout.out_b0_dwi_merge), 
    #("out_updated_bval", workflow.prepare_reference_b0_1.lzout.out_updated_bval), 
    #("out_updated_bvec", workflow.prepare_reference_b0_1.lzout.out_updated_bvec),
    ("out_parameter", workflow.eddy.lzout.out_parameter),
    ("out_corrected", workflow.eddy.lzout.out_corrected),
    ("out_rotated_bvecs", workflow.eddy.lzout.out_rotated_bvecs),
])



In [ ]:
## Workflow execution
from pydra import Submitter

with Submitter(plugin="cf") as submitter:
    submitter(workflow)

results = workflow.result(return_inputs=True)

220610-10:08:43,416 nipype.workflow INFO:
	 [Node] Setting-up "bids_data_grabber" in "/private/var/folders/pf/jc4hflzn2jd71fnht2_2kfm0000w83/T/tmpboapvdzy/Nipype1Task_1dcbacabd20302547437efbbb2ae13a0f479571d8b6629d402171fb38984e797/bids_data_grabber".
220610-10:08:43,422 nipype.workflow INFO:
	 [Node] Executing "bids_data_grabber" <nipype.interfaces.io.BIDSDataGrabber>
220610-10:08:43,844 nipype.workflow INFO:
	 [Node] Finished "bids_data_grabber", elapsed time 0.267325s.
Running pipeline for sub-PREVDEMALS0010025PG | ses-M00 (TotalReadoutTime = 0.0342002, PhaseEncodingDirection = y-)
bvals:  [   0.    0. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
 1000. 1000. 1000. 1000. 1000. 1000.    0. 1000. 1000. 1000. 1000. 1000.
 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.    0. 1000.
 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
 1000. 1000.    0. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
 1000. 1000. 1000. 1000. 1000. 1000

220610-10:09:03,575 nipype.interface INFO:
	 stderr 2022-06-10T10:09:03.574957:WARNING: Both reference and input images have an sform matrix set
220610-10:09:03,575 nipype.interface INFO:
	 stdout 2022-06-10T10:09:03.575639:The output image will use the sform from the reference image
220610-10:09:03,575 nipype.interface INFO:
	 stdout 2022-06-10T10:09:03.575639:The output image will use the transformed sform from the input image
220610-10:09:03,576 nipype.interface INFO:
	 stdout 2022-06-10T10:09:03.575639:Init Matrix = 
220610-10:09:03,576 nipype.interface INFO:
	 stdout 2022-06-10T10:09:03.575639:1.000000 0.000000 0.000000 0.000000 
220610-10:09:03,577 nipype.interface INFO:
	 stdout 2022-06-10T10:09:03.575639:0.000000 1.000000 0.000000 0.000000 
220610-10:09:03,577 nipype.interface INFO:
	 stdout 2022-06-10T10:09:03.575639:0.000000 0.000000 1.000000 0.000000 
220610-10:09:03,577 nipype.interface INFO:
	 stdout 2022-06-10T10:09:03.575639:0.000000 0.000000 0.000000 1.000000 
220610-10

220610-10:09:07,235 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.235000: >> # 4mm scale
220610-10:09:07,235 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.235000: >> setscale 4
220610-10:09:07,339 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.339908: >> setoption smoothing 4
220610-10:09:07,340 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.339908: >> clear U
220610-10:09:07,340 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.339908: >> clear UA 
220610-10:09:07,340 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.339908: >> clear UB
220610-10:09:07,340 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.339908: >> clear US
220610-10:09:07,340 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.339908: >> clear UP
220610-10:09:07,340 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.339908: >> # remeasure costs at this scale
220610-10:09:07,340 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.340403: >> measurecost 7 S 0 0 0 0 0 0 rel
220610-10:09:07,

220610-10:09:07,658 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.658271: >> # also try the identity transform as a starting point at this resolution
220610-10:09:07,658 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.658271: >> clear UQ
220610-10:09:07,658 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.658271: >> setrow UQ  1 0 0 0  0 1 0 0  0 0 1 0  0 0 0 1
220610-10:09:07,658 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.658590: >> optimise 7 UQ  0.0   0.0   0.0   0.0   0.0   0.0   0.0  rel 4
220610-10:09:07,664 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.664598: >> sort U
220610-10:09:07,664 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.664598: >> copy U UA
220610-10:09:07,664 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.664598: >> # select best 4 optimised solutions and try perturbations of these
220610-10:09:07,664 nipype.interface INFO:
	 stdout 2022-06-10T10:09:07.664598: >> clear U
220610-10:09:07,664 nipype.interface INFO:
	 stdout 20

220610-10:09:08,139 nipype.interface INFO:
	 stdout 2022-06-10T10:09:08.139744: >> optimise 7 UA:1-4 -1.0   0.0   0.0   0.0   0.0   0.0   0.0  rel 4
220610-10:09:08,146 nipype.interface INFO:
	 stdout 2022-06-10T10:09:08.146051: >> optimise 7 UA:1-4 -1.0   0.0   0.0   0.0   0.0   0.0   0.0  rel 4
220610-10:09:08,165 nipype.interface INFO:
	 stdout 2022-06-10T10:09:08.165685: >> optimise 7 UA:1-4 -1.0   0.0   0.0   0.0   0.0   0.0   0.0  rel 4
220610-10:09:08,215 nipype.interface INFO:
	 stdout 2022-06-10T10:09:08.215777: >> optimise 7 UA:1-4 -1.0   0.0   0.0   0.0   0.0   0.0   0.0  rel 4
220610-10:09:08,272 nipype.interface INFO:
	 stdout 2022-06-10T10:09:08.272421: >> optimise 7 UA:1-4 -1.0   0.0   0.0   0.0   0.0   0.0   0.0  rel 4
220610-10:09:08,374 nipype.interface INFO:
	 stdout 2022-06-10T10:09:08.374767: >> optimise 7 UA:1-4  0.0   1.0   0.0   0.0   0.0   0.0   0.0  rel 4
220610-10:09:08,388 nipype.interface INFO:
	 stdout 2022-06-10T10:09:08.388565: >> optimise 7 UA:1-4  0.0 

...

220610-10:09:11,588 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.588110: >> copy U UD
220610-10:09:11,588 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.588110: >> setoption boundguess 1
220610-10:09:11,588 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.588428: >> if MAXDOF > 7
220610-10:09:11,588 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.588428: >>  clear U
220610-10:09:11,588 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.588428: >> if MAXDOF > 7
220610-10:09:11,588 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.588428: >>  optimise 9  UD:1  0.0   0.0   0.0   0.0   0.0   0.0   0.0  rel 1
220610-10:09:11,588 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.588428: >> copy U UE
220610-10:09:11,588 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.588428: >> if MAXDOF > 9
220610-10:09:11,588 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.588428: >>  clear U
220610-10:09:11,588 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.588428: >> i

220610-10:09:11,876 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.876808: >> # in addition, try qsform as the final transformation, not just an initialisation
220610-10:09:11,876 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.876808: >> clear UG
220610-10:09:11,877 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.877118: >> setrowqsform UG
220610-10:09:11,877 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.877225: >> measurecost 12 UG:1  0.0   0.0   0.0   0.0   0.0   0.0   0.0  rel 1
220610-10:09:11,879 nipype.interface INFO:
	 stdout 2022-06-10T10:09:11.879681: >> sort U
220610-10:09:12,241 nipype.workflow INFO:
	 [Node] Finished "flirt", elapsed time 8.99053s.
220610-10:09:12,254 nipype.workflow INFO:
	 [Node] Finished "flirt", elapsed time 9.018707s.
220610-10:09:12,359 nipype.workflow INFO:
	 [Node] Finished "flirt", elapsed time 9.139271s.
220610-10:09:12,372 nipype.workflow INFO:
	 [Node] Finished "flirt", elapsed time 9.009829s.
220610-10:09:12,427 nipype.workf

220610-10:09:15,438 nipype.interface INFO:
	 stdout 2022-06-10T10:09:15.438495: >> optimise 7 UP:1-3  0.0   0.0   0.0   0.0   0.0   0.0   0.0  rel 4
220610-10:09:15,449 nipype.interface INFO:
	 stdout 2022-06-10T10:09:15.449888: >> optimise 7 UP:1-3  0.0   0.0   0.0   0.0   0.0   0.0   0.0  rel 4
220610-10:09:15,490 nipype.interface INFO:
	 stdout 2022-06-10T10:09:15.490627: >> # also try the identity transform as a starting point at this resolution
220610-10:09:15,490 nipype.interface INFO:
	 stdout 2022-06-10T10:09:15.490627: >> clear UQ
220610-10:09:15,490 nipype.interface INFO:
	 stdout 2022-06-10T10:09:15.490627: >> setrow UQ  1 0 0 0  0 1 0 0  0 0 1 0  0 0 0 1
220610-10:09:15,491 nipype.interface INFO:
	 stdout 2022-06-10T10:09:15.491046: >> optimise 7 UQ  0.0   0.0   0.0   0.0   0.0   0.0   0.0  rel 4
220610-10:09:15,502 nipype.interface INFO:
	 stdout 2022-06-10T10:09:15.502497: >> # also try the identity transform as a starting point at this resolution
220610-10:09:15,502 nipy

220610-10:09:17,260 nipype.interface INFO:
	 stdout 2022-06-10T10:09:17.260922: >> sort U
220610-10:09:17,261 nipype.interface INFO:
	 stdout 2022-06-10T10:09:17.260922: >> copy U UC
220610-10:09:17,261 nipype.interface INFO:
	 stdout 2022-06-10T10:09:17.260922: >> clear U
220610-10:09:17,261 nipype.interface INFO:
	 stdout 2022-06-10T10:09:17.260922: >> optimise 7  UC:1  0.0   0.0   0.0   0.0   0.0   0.0   0.0  rel 4
220610-10:09:17,262 nipype.interface INFO:
	 stdout 2022-06-10T10:09:17.262064: >> # in addition, try qsform as the final transformation, not just an initialisation
220610-10:09:17,262 nipype.interface INFO:
	 stdout 2022-06-10T10:09:17.262064: >> clear UG
220610-10:09:17,262 nipype.interface INFO:
	 stdout 2022-06-10T10:09:17.262064: >> setrowqsform UG
220610-10:09:17,262 nipype.interface INFO:
	 stdout 2022-06-10T10:09:17.262064: >> measurecost 12 UG:1  0.0   0.0   0.0   0.0   0.0   0.0   0.0  rel 1
220610-10:09:17,262 nipype.interface INFO:
	 stdout 2022-06-10T10:09:17

220610-10:09:20,205 nipype.workflow INFO:
	 [Node] Setting-up "merge" in "/private/var/folders/pf/jc4hflzn2jd71fnht2_2kfm0000w83/T/tmp4_x59b1o/Nipype1Task_8e613948eeae8d99bc21cf54a88615a8defd7ddf994d16208c6400a02ac05782/merge".
220610-10:09:20,210 nipype.workflow INFO:
	 [Node] Executing "merge" <nipype.interfaces.fsl.utils.Merge>
220610-10:09:21,158 nipype.workflow INFO:
	 [Node] Finished "merge", elapsed time 0.946545s.
b0_flirt_results:  ({'b0_flirt_workflow.split_moving': [PosixPath('/private/var/folders/pf/jc4hflzn2jd71fnht2_2kfm0000w83/T/tmpbwh10bzq/Workflow_1199fb24ddf7b5a08e32c9f3a945607441f30aaf978d98aca3ffcddd92a291ec/vol0000.nii.gz'), PosixPath('/private/var/folders/pf/jc4hflzn2jd71fnht2_2kfm0000w83/T/tmpbwh10bzq/Workflow_1199fb24ddf7b5a08e32c9f3a945607441f30aaf978d98aca3ffcddd92a291ec/vol0001.nii.gz'), PosixPath('/private/var/folders/pf/jc4hflzn2jd71fnht2_2kfm0000w83/T/tmpbwh10bzq/Workflow_1199fb24ddf7b5a08e32c9f3a945607441f30aaf978d98aca3ffcddd92a291ec/vol0002.nii.gz'), Po

In [ ]:
results

In [ ]:
workflow.set_output([
    ("image_id", workflow.init_input_node.lzout.image_id),
        ("t1w",workflow.init_input_node.lzout.t1w),
        ("dwi",workflow.init_input_node.lzout.dwi),
        ("bvec",workflow.init_input_node.lzout.bvec),
        ("bval",workflow.init_input_node.lzout.bval),
        ("total_readout_time",workflow.init_input_node.lzout.total_readout_time),
        ("phase_encoding_direction",workflow.init_input_node.lzout.phase_encoding_direction),
])